In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
CONGESTION = 1

SUPPLY_BID = 0
LOAD_ATTACK = 0

LOAD1 = 0
DA_LOAD2 = 75       # in MWs

RT_LOAD2 = 90 + LOAD_ATTACK # in MWs

In [3]:
import os
import copy
import pandas as pd

def input_data(datadir):
    """
    Reading 3-bus data from the CSV files in <datadir> 
    and processing them as dataframes.
    
    return (dataframes): bus, gen, gencost, branch
    """
    
    # Read the CSV files into dataframes
    gen = pd.read_csv(os.path.join(datadir, 'gen.csv'))
    gencost = pd.read_csv(os.path.join(datadir, 'gencost.csv'))
    branch = pd.read_csv(os.path.join(datadir, 'branch.csv'))
    bus = pd.read_csv(os.path.join(datadir, 'bus.csv'))

    # Rename all columns to lowercase
    gen.columns = gen.columns.str.lower()
    gencost.columns = gencost.columns.str.lower()
    branch.columns = branch.columns.str.lower()
    bus.columns = bus.columns.str.lower()

    # Create generator ids
    gen['id'] = range(1, len(gen) + 1)
    gencost['id'] = [1, 2]

    # Create line ids
    branch['id'] = range(1, len(branch) + 1)

    # Calculate the susceptance of each line
    branch['sus'] = 1 / branch['x']
    
    return bus, gen, gencost, branch


# conversion to per-unit from Ohms
baseKV = 230 * 1e3  # 230 kV
mbase  = 100 * 1e6  # 100 MVA
zbase  = ( baseKV * baseKV ) / mbase
x = 0.1
xpu = x/zbase
sus = 1/xpu
print(f"impedance_pu: {xpu}, sus_pu: {sus}")

print("Day Ahead Market(DAM) Data:\n")
DAM_bus, DAM_gen, DAM_gencost, branch = input_data(datadir = 'opf_data')

# modify the data as per the requirements of the current system
print("Updated DAM Bus Dataframe:")
DAM_bus["pd"] = [LOAD1, DA_LOAD2, 0]      # demand on each bus (in MWs)
print(DAM_bus)
print("")

print("Updated DAM_gen:")
DAM_gen.loc[1, "bus"] = 3       # add generator to bus 3 
print(DAM_gen)
print("")

print("Updated DAM_gencost:")
DAM_gencost["x1"] = [0.3, 0.8]  # generator cost curves
DAM_gencost["y1"] = [  3,   8]
print(DAM_gencost)
print("")

# print("Updated DAM_branch:")
branch['x'] = xpu               # in pu via per unit calculations
branch['sus'] = 1 / branch['x']

print("Updated Branch:")
print(branch)

if CONGESTION:
    branch.loc[((branch['fbus'] == 1) & (branch['tbus'] == 2)), 'ratea'] = 47  # creating congestion
#    branch.loc[((branch['fbus'] == 2) & (branch['tbus'] == 3)), 'ratea'] = 47  # creating congestion

impedance_pu: 0.0001890359168241966, sus_pu: 5290.0
Day Ahead Market(DAM) Data:

Updated DAM Bus Dataframe:
   bus_i  type  pd     qd  gs  bs  area  vm  va  basekv  zone  vmax  vmin
0      1     2   0   0.00   0   0     1   1   0     230     1   1.1   0.9
1      2     2  75   0.00   0   0     1   1   0     230     1   1.1   0.9
2      3     1   0  98.61   0   0     1   1   0     230     1   1.1   0.9

Updated DAM_gen:
   bus   pg  qg   qmax   qmin  vg  mbase  status  pmax  pmin  ...  qc1min  \
0    1   40   0   30.0  -30.0   1    100       1  1000     0  ...       0   
1    3  170   0  127.5 -127.5   1    100       1  1000     0  ...       0   

   qc1max  qc2min  qc2max  ramp_agc  ramp_10  ramp_30  ramp_q  apf  id  
0       0       0       0         0        0        0       0    0   1  
1       0       0       0         0        0        0       0    0   2  

[2 rows x 22 columns]

Updated DAM_gencost:
   model  startup  shutdown  n   x1  y1  id
0      2        0         0  2  0.3   

### Construct all the relevant matrices from the above dataframes for optimization problem formulation (Overall approach to Construction):

#### Optimization variable:
- Ensure the dimension of `p = [x, y, v]` is fixed.
- Both the below decisions to easily compute `p_{-i}`
    -  Make `x` dynamically sized - `len(G)`
    -  Make `y` and `v` constant sized - `len(N)` each
    
#### Objective Function:
- (Done) `A`: Coefficient matrix for the constraints
- (Done) `b`: Right-hand side vector for the constraints

#### Modeling Constraints:
- (Done) `S`: System matrix, possibly including incidence matrix `Inci` and matrix `B`
- (Done) `c`: Capacity bounds associated with the constraint modeled as `lcb` and `ucb`
- `\Phi`: Hardcode based on the modeling choice of variable `p`

In [4]:
import numpy as np

# Modeling Objective Function constants
G_DAM = DAM_gen['id'].values      # 1, 2
N = DAM_bus['bus_i'].values   # all buses in the network: 1, 2, 3

dim_p = len(G_DAM) + len(N) + len(N)

# Modeling Objective Function Constants (A, b):
A = np.zeros( ( dim_p, dim_p ) ) # initialize A as zeros # make the genreators dynamic
b = np.zeros( dim_p )

for i in G_DAM:  # go through each generator
    alpha_i  = DAM_gencost.loc[DAM_gencost['id']==i, 'x1'].values[0]   # alpha_i associated 
    beta_i   = DAM_gencost.loc[DAM_gencost['id']==i, 'y1'].values[0]   #  beta_i associated
    
    bus_index = DAM_gen.loc[DAM_gen['id']==i, 'bus'].values[0]  # bus index associated with curr generator
    
    A[    i-1,     i-1] = alpha_i   # for physical supply (0, 1, 2)
    #A[len(G_DAM) + len(N) + bus_index-1, len(G_DAM) + len(N) + bus_index-1] = alpha_i   # for physical supply (6, 7, 8) - hard coded for the 3-bus
    
    b[    i-1] = beta_i
    #b[len(G_DAM) + len(N) + bus_index-1] = beta_i   # for physical supply (6, 7, 8) - hard coded for the 3-bus

print( A )
print( b )

[[0.3 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]
[3. 8. 0. 0. 0. 0. 0. 0.]


In [5]:
# Modeling Contraints (S, c, \Phi):

# construct B in N x N 
B = np.zeros( ( len(N), len(N) ) ) # initialize B as zeors
for i in N:  # go through each bus:
    #print(i)
    tbus_values = branch.loc[branch['fbus'] == i, 'tbus'].values  # find all the other buses connected to this bus and their sus value
    #print(tbus_values)
    for curr_to_bus in tbus_values:
        curr_sus = branch.loc[(branch['fbus'] == i) & (branch['tbus'] == curr_to_bus), 'sus'].values[0]
        #print(curr_sus)
        B[i-1, curr_to_bus-1] = 0 - curr_sus
        B[curr_to_bus-1, i-1] = 0 - curr_sus
        
    curr_index = i-1
    curr_vector = B[curr_index]
    curr_total = sum(curr_vector[:curr_index]) + sum(curr_vector[curr_index+1:]) # sum of all except curr_index
    B[i-1, i-1] =  0 - curr_total # to be the sum of the remaining elements in   
    #print(B) 
    #print("")
B_plus = np.linalg.pinv(B)   # Compute B^+ for computing downstream power-flows
print("Susceptance Matrix:")
print(B) # in N x N 
print("Pseudo-inverse of B:")
print(B_plus)


# construct Incidence in L x N
ucb = []  
lcb = []
incidence = []
for l in branch.index:
    from_bus = branch.loc[l, 'fbus']
    to_bus = branch.loc[l, 'tbus']
    curr_sus = branch.loc[(branch['fbus'] == from_bus) & (branch['tbus'] == to_bus), 'sus'].values[0]
    
    vector = np.zeros( len(N) ) # Create vector for this branch
    vector[from_bus - 1] =     curr_sus
    vector[to_bus - 1]   = 0 - curr_sus
    
    incidence.append(vector)     
    ucb.append(     branch.loc[l, 'ratea'] )
    lcb.append( 0 - branch.loc[l, 'ratea'] )
    
incidence = np.array(incidence)        # convert to numpy
ucb = np.array(ucb).reshape(-1, 1)  
lcb = np.array(lcb).reshape(-1, 1)  

print("Incidence Matrix incidence:")
print(incidence)
print("Capacity Bounds:")
print(ucb, lcb)

# S in L x N = incidence * B_plus (as per the paper)
S = np.dot(incidence, B_plus)    
print("Shift Factor Matrix S:")
print(S, S.shape)

# construct \Phi (hardcoded for now)
#  -------------- x ---- y -------- v -------
PHI = np.array([ [1, 0,  1,  0,  0, 1, 0, 0],
                 [0, 0,  0,  1,  0, 0, 1, 0],
                 [0, 1,  0,  0,  1, 0, 0, 1]  ])

print(PHI, PHI.shape)

Susceptance Matrix:
[[10580. -5290. -5290.]
 [-5290. 10580. -5290.]
 [-5290. -5290. 10580.]]
Pseudo-inverse of B:
[[ 4.20079815e-05 -2.10039908e-05 -2.10039908e-05]
 [-2.10039908e-05  4.20079815e-05 -2.10039908e-05]
 [-2.10039908e-05 -2.10039908e-05  4.20079815e-05]]
Incidence Matrix incidence:
[[ 5290.     0. -5290.]
 [ 5290. -5290.     0.]
 [    0.  5290. -5290.]]
Capacity Bounds:
[[500]
 [ 47]
 [500]] [[-500]
 [ -47]
 [-500]]
Shift Factor Matrix S:
[[ 3.33333333e-01  7.80218988e-17 -3.33333333e-01]
 [ 3.33333333e-01 -3.33333333e-01  2.42522473e-17]
 [ 3.63885354e-17  3.33333333e-01 -3.33333333e-01]] (3, 3)
[[1 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 1 0]
 [0 1 0 0 1 0 0 1]] (3, 8)


## DAM Optimization (Vectorized format):

Objective function:
$$
\min \quad 0.5 p^T A p + b^T p
$$

Power balance constraint:
$$
\quad \quad \quad \quad \quad 1^T p = 0 \quad : \lambda \in \mathbb{R}
$$ 

Line constraints:
$$
\quad \quad \quad \quad \quad -c \leq S \Phi p \leq c \quad : \mu^-, \mu^+ \in \mathbb{R}^{L}
$$

Generation limits:
$$
\quad \quad \quad \quad \quad p^{min} \leq p \leq p^{max} \in \mathbb{R}^{G_{DAM}}
$$

the optimization variable are cleared energy of physical, defined as:
$$
p \triangleq [x, y, v] ^ T
$$

and the dimensions are:
$$
A \in \mathbb{R}^{p_{dim}\times p_{dim}},\;b \in \mathbb{R}^{p_{dim}} \quad p_{dim} = G_{DAM} + N + N
$$
$$x\in \mathbb{R}^{G_{DAM}} \;- \text{Physical Supply Variable} $$ 
$$y\in \mathbb{R}^{N} \;- \text{Physical Load Constant} $$ 
$$v\in \mathbb{R}^{N} \;- \text{Virtual Supply bid Constant} $$
$$S \in \mathbb{R}^{L \times N} \;-\quad \text{Shift Factor Matrix}$$
$$\Phi \in \mathbb{R}^{N \times p_{dim}} $$

In [6]:
import cvxpy as cp

def DAM_dcopf_vectorized(gen, branch, gencost, bus, S, ucb, lcb):
    
    # Define base MVA (for conversion between per unit and MWs)
    baseMVA = gen['mbase'].iloc[0]
    
    x = cp.Variable( len(G_DAM) )      # dynamic based on generator locations
    y = 0 - bus["pd"].to_numpy()   # dynamically read load data from dataframe - negative for load values
    v = np.array([0, SUPPLY_BID, 0])        # manually change to add virtual bids
        
    # decision variable in MWs
    p = cp.Variable( dim_p )

    # Define constraints on the concatenated variable - dim_p
    constraints = [ p[0:len(G_DAM)] == x,
                    p[len(G_DAM):len(G_DAM)+len(y)] == y,
                    p[len(G_DAM)+len(y) :] == v       ]
    
    objective = cp.Minimize( 0.5 * cp.quad_form(p, A) + cp.matmul(b, p) ) # 1a
    
    # constraint - 1 (equality constraints)
    ones_vector = np.ones( dim_p )  # A vector of ones with the same length as p
    constraints += [ cp.matmul(ones_vector, p) == 0 ]  # Adding the balance constraint
    
    def find_flows(p):
        p_normalized = p / baseMVA               # in per unit
        pp_vector = np.matmul(PHI, p_normalized)
        flows = np.matmul(S, pp_vector)           # power-flow vector of size L - still in per unit
        flows_scaled = baseMVA * flows             # power-flow vector of size L - in MWs
        flows_scaled = np.reshape(flows_scaled, (-1, 1))
        return flows_scaled
    
    # constraint - 2 (inequality capacity constraints)
    flows_scaled = find_flows(p)
    constraints.append(
        flows_scaled <= ucb
    )
    constraints.append(
        flows_scaled >= lcb
    )
    
    # constraint - 3 (inequality Max, Min generation constraints)
    for g in G_DAM: # 1, 2
        constraints.append( p[g-1] <= gen.loc[g-1, 'pmax'] ) # G1-g[0] AND G3-g[2] only 
    for g in G_DAM: # 1, 2
        constraints.append( p[g-1] >= gen.loc[g-1, 'pmin'] ) # G1-g[0] AND G3-g[2] only
        
    # Define the problem and solve
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.OSQP)     # is an open-source C library for solving convex quadratic programs
    #problem.solve(solver=cp.XPRESS ) 
    
    # Access the optimized values
    optimized_p = p.value
    #print(optimized_p)

    # Prepare generation output
    optimized_x = optimized_p[0:len(G_DAM)]
    generation = pd.DataFrame({
        'id':   gen['id'],
        'node': gen['bus'],
        'gen':  optimized_x
    })
    
    # prepare power-flow output
    optimized_flows = find_flows(optimized_p)
    flows = pd.DataFrame({
        'fbus': branch['fbus'],
        'tbus': branch['tbus'],
        'flow': np.squeeze(optimized_flows)
    })
    
    # prepare prices
#     for constraint in constraints[3 : 5+1]:
#         print(constraint.dual_value) 
        
    llambda  = abs( constraints[3].dual_value ) # not sure if abs is correct or flipping?
    mu_plus  = constraints[4].dual_value
    mu_minus = constraints[5].dual_value
    mu = mu_plus - mu_minus
    ones_vector = np.ones( len(N) ).reshape(-1, 1)
    pi = llambda * ones_vector - np.matmul(S.T, mu)
    # Extract the prices (dual values of the balance constraints)
    prices = pd.DataFrame({
        'node': bus['bus_i'],
        'value': np.squeeze(pi),
        '  mu_plus': np.squeeze(mu_plus),
        '  mu_minus': np.squeeze(mu_minus),
        '  mu(dual)': np.squeeze(mu)
    })
            
    # Return the solution and objective value
    return {
        'generation': generation.round(7),
        'flows': flows.round(5),
        'prices': prices,
        'p': optimized_p,
        'cost': '{:.5f}'.format(problem.value),
        'status': problem.status
    }

/opt/homebrew/Caskroom/miniforge/base/envs/PGOC/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
print("Day Ahead Market(DAM) Results:\n")

DAM = DAM_dcopf_vectorized(DAM_gen, branch, DAM_gencost, DAM_bus, S, ucb, lcb)
for key, value in DAM.items():
    if key=="generation":
        optimized_x = value["gen"].to_numpy()
    if key=="p":
        p = value
        #p = np.where(np.abs(p) < 1e-2, 0, p)
    if key=="prices":
        mu = value["  mu(dual)"].to_numpy()
    print(f"{key}:") 
    print(value)
    print()
    
print(f"optimized_x = {optimized_x}") # optimized vector to be used in DAM

Day Ahead Market(DAM) Results:

generation:
   id  node        gen
0   1     1  59.090909
1   2     3  15.909091

flows:
   fbus  tbus      flow
0     1     3  14.39394
1     1     2  44.69697
2     2     3 -30.30303

prices:
   node      value    mu_plus    mu_minus    mu(dual)
0     1  20.727273        0.0         0.0         0.0
1     2  20.727273        0.0         0.0         0.0
2     3  20.727273        0.0         0.0         0.0

p:
[ 5.90909091e+01  1.59090909e+01 -1.60405221e-20 -7.50000000e+01
 -1.60405221e-20 -1.60405221e-20 -1.60405221e-20 -1.60405221e-20]

cost:
929.54545

status:
optimal

optimized_x = [59.0909091 15.9090909]


### Post Optimality Analysis:

$$p_{-i} \triangleq [x, y, v_{-i}] ^ T \quad \quad p_{-i_{dim}}$$
$$\bar D \in \mathbb{R}^{\bar L \times L} \;$$
$$S \in \mathbb{R}^{L \times N}$$
$$\Phi_{-i} \in \mathbb{R}^{N \times {p_{-i}}_{dim}}$$
$$A_{-i} \in \mathbb{R}^{{p_{-i}}_{dim} \times {p_{-i}}_{dim}} $$
 
$$ \bar D S \in \mathbb{R}^{\bar L \times N}$$
$$X = \bar D S \Phi_{-i}\in \mathbb{R}^{\bar L \times {p_{-i}}_{dim}} \;$$

In [8]:
p_minusi   = np.concatenate((p[:-2], p[-1:]))
p_minusi_dim = len(p_minusi)
PHI_minusi = np.delete(PHI, -2, axis=1)
A_minusi = np.delete(np.delete(A, -2, axis=0), -2, axis=1) # delete row and column

ppvector_minusi = np.matmul(PHI_minusi, p_minusi)
flows = np.matmul(S, ppvector_minusi)

print(f"p_minusi:\n{p_minusi}")
print(f"PHI_minusi:\n{PHI_minusi}")
print(ppvector_minusi)
print(f"flows:\n{flows}")

Lbar = np.count_nonzero(mu)  # how many non-zero entries in mu? - Lbar
nonzero_indices_mu = np.nonzero(mu) # find the index of non-zero in mu? - using which construct 
L = incidence.shape[0]
Dbar = np.zeros((Lbar, L))
Dbar[np.arange(Lbar), nonzero_indices_mu[0]] = 1
print(f"Dbar: {Dbar} with shape {Dbar.shape}")

X = np.matmul(Dbar, np.matmul(S, PHI_minusi))
print(f"X: {X} of Shape {X.shape}")

LAMBDA = np.where(A_minusi != 0, 1 / A_minusi, 0)
#print(LAMBDA, LAMBDA.shape)

ones_vector_p = np.ones( p_minusi_dim ).reshape(-1, 1) # column vector
H_left  = np.vstack( (ones_vector_p.T,  X) )
H_right = np.vstack( (ones_vector_p.T, -X) ).T
print(H_left.shape, H_right.shape)
H = np.matmul(H_left, np.matmul(LAMBDA, H_right))
print(f"H: {H} of Shape {H.shape}")

H_inverse = np.linalg.inv(H)
print(f"H_inverse: {H_inverse} of Shape {H_inverse.shape}")

ones_vector_N = np.ones( len(N) ).reshape(-1, 1)        # column vector
temp3 = np.vstack( (-ones_vector_N.T, np.matmul(Dbar, S)) ).T
print(temp3, temp3.shape)

zeros_vector_Lbar = np.zeros( Lbar ).reshape(-1, 1)      # column vector
DAM_sesitivity_right = np.vstack( (1, zeros_vector_Lbar) )
print(DAM_sesitivity_right, DAM_sesitivity_right.shape)

DAM_sesitivity = np.matmul(temp3, np.matmul(H_inverse, DAM_sesitivity_right))

print(f"\nDAM_sesitivity:\n {DAM_sesitivity}")

p_minusi:
[ 5.90909091e+01  1.59090909e+01 -1.60405221e-20 -7.50000000e+01
 -1.60405221e-20 -1.60405221e-20 -1.60405221e-20]
PHI_minusi:
[[1 0 1 0 0 1 0]
 [0 0 0 1 0 0 0]
 [0 1 0 0 1 0 1]]
[ 59.09090909 -75.          15.90909091]
flows:
[ 14.39393939  44.6969697  -30.3030303 ]
Dbar: [] with shape (0, 3)
X: [] of Shape (0, 7)
(1, 7) (7, 1)
H: [[4.58333333]] of Shape (1, 1)
H_inverse: [[0.21818182]] of Shape (1, 1)
[[-1.]
 [-1.]
 [-1.]] (3, 1)
[[1.]] (1, 1)

DAM_sesitivity:
 [[-0.21818182]
 [-0.21818182]
 [-0.21818182]]


<ipython-input-8-319e7463f7e4>:24: RuntimeWarning: divide by zero encountered in divide
  LAMBDA = np.where(A_minusi != 0, 1 / A_minusi, 0)


In [9]:
print("Real Time Market(RTM) Data:\n")

datadir = 'opf_data'
RTM_bus, RTM_gen, RTM_gencost, _ = input_data(datadir)

print("Updated Bus Dataframe:")
RTM_bus["pd"] = [LOAD1, RT_LOAD2, 0]  # remember this is positive. Keep in mind the sign when using later.
print(RTM_bus)
print("")

print("Updated RTM_gen:")
# Create a DataFrame with the new row data
new_row = pd.DataFrame({'bus': [3], 'pg': [100], 'qg': [0], 'qmax': [0], 'qmin': [0], 'vg': [0], 'mbase': [100], 'status': [1], 'pmax': [1000], 'pmin': [0], 'qc1max': [0], 'qc1min': [0], 'qc2max': [0], 'qc2min': [0], 'ramp_agc': [0], 'ramp_10': [0], 'ramp_30': [0], 'ramp_q': [0], 'apf': [0], 'id': [3]})
RTM_gen = pd.concat([RTM_gen, new_row], ignore_index=True) # Append the new row to RTM_gen
print(RTM_gen)
print("")

print("Updated RTM_gencost:")
last_row = RTM_gencost.iloc[-1] # Get the last row of the DataFrame
RTM_gencost = pd.concat([RTM_gencost, pd.DataFrame([last_row])], ignore_index=True) # Concatenate the original DataFrame with the duplicated row
RTM_gencost["x1"] = [1.8, 1.7,  0.1]
RTM_gencost["y1"] = [ 10,   5,   14]
RTM_gencost["id"] = [  1,   2,    3]
print(RTM_gencost)
print("")

print("Updated RTM_branch:")
print(branch)

Real Time Market(RTM) Data:

Updated Bus Dataframe:
   bus_i  type  pd     qd  gs  bs  area  vm  va  basekv  zone  vmax  vmin
0      1     2   0   0.00   0   0     1   1   0     230     1   1.1   0.9
1      2     2  90   0.00   0   0     1   1   0     230     1   1.1   0.9
2      3     1   0  98.61   0   0     1   1   0     230     1   1.1   0.9

Updated RTM_gen:
   bus   pg  qg   qmax   qmin  vg  mbase  status  pmax  pmin  ...  qc1min  \
0    1   40   0   30.0  -30.0   1    100       1  1000     0  ...       0   
1    2  170   0  127.5 -127.5   1    100       1  1000     0  ...       0   
2    3  100   0    0.0    0.0   0    100       1  1000     0  ...       0   

   qc1max  qc2min  qc2max  ramp_agc  ramp_10  ramp_30  ramp_q  apf  id  
0       0       0       0         0        0        0       0    0   1  
1       0       0       0         0        0        0       0    0   2  
2       0       0       0         0        0        0       0    0   3  

[3 rows x 22 columns]

Updated R

### Construct all the relevant matrices from the above dataframes for optimization problem formulation (Overall approach to Construction):

#### Optimization variable:
- Ensure the dimension of `z` is fixed.
    
#### Objective Function:
- (Done) `C`: Coefficient matrix for the constraints
- (Done) `d`: Right-hand side vector for the constraints

#### Modeling Constraints:
- `l` for 
- `\Psi` for `x` from day ahead
- `\Theta` for `z` from day ahead
- `\Omega` for `l` from day ahead

In [10]:
import numpy as np

# Modeling Objective Function constants
G_RTM = RTM_gen['id'].values      # 1, 2, 3

dim_x = len(optimized_x)
dim_z = len(G_RTM)

# Modeling Objective Function Constants (A, b):
C = np.zeros( ( dim_z, dim_z ) ) # initialize A as zeros # make the genreators dynamic
d = np.zeros( dim_z )

for i in G_RTM:  # go through each generator
    alpha_i  = RTM_gencost.loc[RTM_gencost['id']==i, 'x1'].values[0]   # alpha_i associated 
    beta_i   = RTM_gencost.loc[RTM_gencost['id']==i, 'y1'].values[0]   #  beta_i associated
    bus_index = RTM_gen.loc[RTM_gen['id']==i, 'bus'].values[0]  # bus index associated with curr generator
    
    C[    i-1,     i-1] = alpha_i   # for physical supply (0, 1, 2)
    d[    i-1] = beta_i

print( C )
print( d )

[[1.8 0.  0. ]
 [0.  1.7 0. ]
 [0.  0.  0.1]]
[10.  5. 14.]


In [11]:
# for x
PSI = np.array([ [1, 0],
                 [0, 0],
                 [0, 1]  ])
# for z
THETA = np.array([  [1, 0, 0],
                    [0, 1, 0],
                    [0, 0, 1]  ])
# for l
OMEGA = np.array([  [1, 0, 0],
                    [0, 1, 0],
                    [0, 0, 1]  ])

## Real-Time Market Optimization (Vectorized format):

$$
\min \quad 0.5 z^T C z + d^T z
$$

Power balance constraint:
$$
1^T z + 1^T x = 1^T l \quad : \delta
$$

Line constraints:
$$
-c \leq S(\Psi x + \Theta z - \Omega l) \leq c \quad : \eta^-, \eta^+
$$

Generation limits:
$$
z^{min} \leq z \leq z^{max}
$$

the optimization variable are adjustment to physical energy:
$$
z 
$$

and the dimensions are:
$$
C \in \mathbb{R}^{z_{dim}\times z_{dim}},\;d \in \mathbb{R}^{z_{dim}} \quad z_{dim} = G_{RTM}
$$
$$z\in \mathbb{R}^{G_{RTM}} \;- \text{Physical Supply Adjustment Variable} $$ 
$$\Theta \in \mathbb{R}^{N \times G_{RTM}} $$

$$x\in \mathbb{R}^{G_{DAM}} \;- \text{Physical Supply Variable} $$ 
$$\Psi \in \mathbb{R}^{N \times G_{DAM}} $$

$$l\in \mathbb{R}^{N} \;- \text{Physical Supply Variable} $$ 
$$\Omega \in \mathbb{R}^{N \times N} $$

$$S \in \mathbb{R}^{L \times N} \;-\quad \text{Shift Factor Matrix}$$

In [12]:
import cvxpy as cp

def RTM_dcopf_vectorized(gen, branch, gencost, bus, S, ucb, lcb, optimized_x):
    
    # Define base MVA (for conversion between per unit and MWs)
    baseMVA = gen['mbase'].iloc[0]
    
    # decision variable in MWs
    z = cp.Variable( len(G_RTM) )      # dynamic based on generator locations
    load_RTM = bus["pd"].to_numpy()    # dynamically read load data from dataframe - negative for load values
    dim_load = len(load_RTM)
            
    objective = cp.Minimize( 0.5 * cp.quad_form(z, C) + cp.matmul(d, z) ) # 6a
    
    # constraint - 1 (equality constraints)
    ones_vector_z = np.ones( dim_z )
    ones_vector_x = np.ones( dim_x )
    ones_load_RTM = np.ones( dim_load )
    constraints   = [ cp.matmul(ones_vector_z, z) + cp.matmul(ones_vector_x, optimized_x) == cp.matmul(ones_load_RTM, load_RTM) ]  # Adding the balance constraint
    
    def find_flows(z, optimized_x, load_RTM):
        z_normalized = z / baseMVA               # in per unit
        x_normalized = optimized_x / baseMVA               # in per unit
        load_normalized = load_RTM / baseMVA               # in per unit
        
        pp_vector  = np.matmul(THETA,  z_normalized) 
        pp_vector += np.matmul(PSI,    x_normalized)
        pp_vector  -= np.matmul(OMEGA, load_normalized)
        flows = np.matmul(S, pp_vector)           # power-flow vector of size L - still in per unit
        flows_scaled = baseMVA * flows            # power-flow vector of size L - in MWs
        flows_scaled = np.reshape(flows_scaled, (-1, 1))
        return flows_scaled
    
    # constraint - 2 (inequality capacity constraints)
    flows_scaled = find_flows(z, optimized_x, load_RTM)
    constraints.append(
        flows_scaled <= ucb
    )
    constraints.append(
        flows_scaled >= lcb
    )
    
    # constraint - 3 (inequality Max, Min generation constraints)
    for g in G_RTM: # 1, 2
        constraints.append( z[g-1] <= gen.loc[g-1, 'pmax'] ) # G1-g[0] AND G3-g[2] only
        
    # Define the problem and solve
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.OSQP)     # is an open-source C library for solving convex quadratic programs
    #problem.solve(solver=cp.XPRESS ) 
    
    # Access the optimized values
    optimized_z = z.value
    #print(optimized_z)

    # Prepare generation output
    generation = pd.DataFrame({
        'id':   gen['id'],
        'node': gen['bus'],
        'gen':  optimized_z
    })
    
    # prepare power-flow output
    optimized_flows = find_flows(optimized_z, optimized_x, load_RTM)
    flows = pd.DataFrame({
        'fbus': branch['fbus'],
        'tbus': branch['tbus'],
        'flow': np.squeeze(optimized_flows)
    })
    
    # prepare prices
#     for constraint in constraints[3 : 5+1]:
#         print(constraint.dual_value) 
        
    delta  = abs( constraints[0].dual_value )
    eta_plus  = constraints[1].dual_value
    eta_minus = constraints[2].dual_value
    eta = eta_plus - eta_minus
    ones_vector = np.ones( len(N) ).reshape(-1, 1)
    sigma = delta * ones_vector - np.matmul(S.T, eta)
    # Extract the prices (dual values of the balance constraints)
    prices = pd.DataFrame({
        'node': bus['bus_i'],
        'value': np.squeeze(sigma),
        '  eta_plus': np.squeeze(eta_plus),
        '  eta_minus': np.squeeze(eta_minus),
        '  eta(dual)': np.squeeze(eta)
        
    })
            
    # Return the solution and objective value
    return {
        'generation': generation.round(7),
        'flows': flows.round(5),
        'prices': prices,
        'z': optimized_z,
        'cost': '{:.5f}'.format(problem.value),
        'status': problem.status
    }

In [13]:
print("Real Time Market(RTM):\n")

RTM = RTM_dcopf_vectorized(RTM_gen, branch, RTM_gencost, RTM_bus, S, ucb, lcb, optimized_x)

for key, value in RTM.items():
    if key=="prices":
        eta = value["  eta(dual)"].to_numpy()
    print(f"{key}:") 
    print(value)
    print()

Real Time Market(RTM):

generation:
   id  node       gen
0   1     1  0.160839
1   2     2  8.251748
2   3     3  6.587413

flows:
   fbus  tbus      flow
0     1     3  12.25175
1     1     2  47.00000
2     2     3 -34.74825

prices:
   node      value    eta_plus    eta_minus    eta(dual)
0     1  10.289510    0.000000          0.0     0.000000
1     2  19.027972   13.107692          0.0    13.107692
2     3  14.658741    0.000000          0.0     0.000000

z:
[0.16083916 8.25174826 6.58741259]

cost:
195.16154

status:
optimal



### Post Optimality Analysis (VB Sensitivity):

$$\bar R \in \mathbb{R}^{\bar L \times L} \;$$
$$S \in \mathbb{R}^{L \times N}$$
$$\hat \Theta \in \mathbb{R}^{N \times G_{RTM}} $$
 
$$ \bar R S \in \mathbb{R}^{\bar L \times N}$$
$$Y = \bar R S \hat \Theta\in \mathbb{R}^{\bar L \times G_{RTM}} \;$$

$$ \bar R S \in \mathbb{R}^{\bar L \times N}$$
$$\Psi \in \mathbb{R}^{N \times G_{DAM}} $$
$$ \bar R S \Psi \in \mathbb{R}^{\bar L \times G_{DAM}}$$

In [14]:
Lbar = np.count_nonzero(eta)  # how many non-zero entries in mu? - Lbar
nonzero_indices_eta = np.nonzero(eta) # find the index of non-zero in mu? - using which construct 
Rbar = np.zeros((Lbar, L))
Rbar[np.arange(Lbar), nonzero_indices_eta[0]] = 1
print(f"Rbar: {Rbar} of shape {Rbar.shape}")

Y = np.matmul(Rbar, np.matmul(S, THETA))
print(f"Y: {Y} of Shape {Y.shape}")

GAMMA = np.where(C != 0, 1 / C, 0)
print(GAMMA, GAMMA.shape)

ones_vector_z = np.ones( dim_z ).reshape(-1, 1)
temp1 = np.vstack((ones_vector_z.T, Y))
temp2 = np.vstack((ones_vector_z.T, -Y)).T
print(temp1.shape, temp2.shape)
E = np.matmul(temp1, np.matmul(GAMMA, temp2))
print(f"E: {E} of Shape {E.shape}")

E_inverse = np.linalg.inv(E)
print(f"E_inverse: {E_inverse} of Shape {E_inverse.shape}")


ones_vector_N = np.ones( len(N) ).reshape(-1, 1)
temp3 = np.vstack( (ones_vector_N.T, -np.matmul(Rbar, S)) ).T
print(temp3, temp3.shape)

ones_vector_DAM = np.ones( len(G_DAM) ).reshape(-1, 1)
temp4 = np.vstack( (ones_vector_DAM.T, np.matmul(np.matmul(Rbar, S), PSI)) )
print(temp4, temp4.shape)

firstPart =  np.matmul(np.matmul(temp3, E_inverse), temp4)
print(firstPart.shape)
secondPart = np.matmul(np.matmul(LAMBDA, H_right), np.matmul(H_inverse, DAM_sesitivity_right))
print(secondPart.shape)

Khat = np.array([[1, 0, 0, 0, 0, 0, 0],
                 [0, 1, 0, 0, 0, 0, 0]])  # Example K matrix

RTM_sesitivity = np.matmul(firstPart, np.matmul(Khat, secondPart))
print(f"\nRTM_sesitivity:\n {RTM_sesitivity}")

Rbar: [[0. 1. 0.]] of shape (1, 3)
Y: [[ 3.33333333e-01 -3.33333333e-01  2.42522473e-17]] of Shape (1, 3)
[[ 0.55555556  0.          0.        ]
 [ 0.          0.58823529  0.        ]
 [ 0.          0.         10.        ]] (3, 3)
(2, 3) (3, 2)
E: [[ 1.11437908e+01  1.08932462e-02]
 [-1.08932462e-02 -1.27087872e-01]] of Shape (2, 2)
E_inverse: [[ 8.97435897e-02  7.69230769e-03]
 [-7.69230769e-03 -7.86923077e+00]] of Shape (2, 2)
[[ 1.00000000e+00 -3.33333333e-01]
 [ 1.00000000e+00  3.33333333e-01]
 [ 1.00000000e+00 -2.42522473e-17]] (3, 2)
[[1.00000000e+00 1.00000000e+00]
 [3.33333333e-01 2.42522473e-17]] (2, 2)
(3, 2)
(7, 1)

RTM_sesitivity:
 [[ 0.73006993]
 [-0.54685315]
 [ 0.09160839]]


<ipython-input-14-8387614e4752>:10: RuntimeWarning: divide by zero encountered in divide
  GAMMA = np.where(C != 0, 1 / C, 0)


In [15]:
sesitivity_VB = DAM_sesitivity - RTM_sesitivity
sesitivity_VB

array([[-0.94825175],
       [ 0.32867133],
       [-0.30979021]])

In [16]:
RTM['prices']['value'] = pd.to_numeric(RTM['prices']['value'])
DAM['prices']['value'] = pd.to_numeric(DAM['prices']['value'])

count = 1
for dam_price, rtm_price in zip(DAM['prices']['value'], RTM['prices']['value']):
    print(f"Bus {count}: {dam_price}, {rtm_price}, {dam_price - rtm_price:.5f}")
    if count == 2:
        if SUPPLY_BID>0:
            print(f"Bus-2 Profit: {(dam_price - rtm_price)*SUPPLY_BID - (LOAD_ATTACK)*rtm_price}")
    count += 1

Bus 1: 20.727272727272727, 10.28951048153846, 10.43776
Bus 2: 20.727272727272727, 19.027972035897502, 1.69930
Bus 3: 20.727272727272727, 14.658741258717981, 6.06853
